 ## Optical Flow in rocAL

This example presents a rocAL video pipeline that loads calculate the optical flow for a given sequence of Frames. Illustrated below how to create a pipeline, set_outputs, build, run the pipeline and enumerate over the results.

 ## Common Code

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch
from amd.rocal.pipeline import Pipeline
import amd.rocal.fn as fn
import amd.rocal.types as types
import numpy as np
import os
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation as animation


/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


## Configuring rocAL pipeline
Configure the pipeline paramters as required by the user.

In [2]:
video_path = "/dockerx/ak_optical_flow/labelled_video/"
_rocal_cpu =  False
batch_size = 10
display = False
num_threads = 1
random_seed = 1
tensor_format = types.NFHWC 
tensor_dtype = types.FLOAT
local_rank = 0
sequence_length=2
n_iter=6

## Defining and Running the Pipeline
Creating the pipeline using video readers for reading the video data.In this pipeline we add cascaded augmentation on the decoded sequence. We enable the output for differnet augmentaion using set_output



In [3]:
class ROCALVideoIterator(object):
    """
    ROCALVideoIterator for pyTorch.

    Parameters
    ----------
    pipelines : list of amd.rocal.pipeline.Pipeline
                List of pipelines to use
    size : int
           Epoch size.
    """

    def __init__(self, pipelines, tensor_layout=types.NCHW, reverse_channels=False, multiplier=None, offset=None, tensor_dtype=types.FLOAT, display=False, sequence_length=3):

        try:
            assert pipelines is not None, "Number of provided pipelines has to be at least 1"
        except Exception as ex:
            print(ex)

        self.loader = pipelines
        self.tensor_format = tensor_layout
        self.multiplier = multiplier if multiplier else [1.0, 1.0, 1.0]
        self.offset = offset if offset else [0.0, 0.0, 0.0]
        self.reverse_channels = reverse_channels
        self.tensor_dtype = tensor_dtype
        self.batch_size = self.loader._batch_size
        self.rim = self.loader.get_remaining_images()
        self.display = display
        self.iter_num = 0
        self.sequence_length = sequence_length
        print("____________REMAINING IMAGES____________:", self.rim)
        self.output = self.dimensions = self.dtype = None

    def next(self):
        return self.__next__()

    def __next__(self):
        if (self.loader.is_empty()):
            raise StopIteration

        if self.loader.rocal_run() != 0:
            raise StopIteration
        else:
            self.output_tensor_list = self.loader.get_output_tensors()
        self.iter_num += 1
        # Copy output from buffer to numpy array
        if self.output is None:
            self.dimensions = self.output_tensor_list[0].dimensions()
            self.dtype = self.output_tensor_list[0].dtype()
            self.layout = self.output_tensor_list[0].layout()
            self.output = np.empty(
                (self.dimensions[0]*self.dimensions[1], self.dimensions[2], self.dimensions[3], self.dimensions[4]), dtype=self.dtype)
        self.output_tensor_list[0].copy_data(self.output)
        img = torch.from_numpy(self.output)
        # Display Frames in a video sequence
        if self.display:
            for batch_i in range(self.batch_size):
                draw_frames(img[batch_i], batch_i, self.iter_num, self.layout)
        return img

    def reset(self):
        self.loader.rocal_reset_loaders()

    def __iter__(self):
        return self

    def __del__(self):
        self.loader.rocal_release()


In [4]:
pipe = Pipeline(batch_size=batch_size, num_threads=num_threads, device_id=local_rank, seed=random_seed, rocal_cpu=_rocal_cpu,
                    tensor_layout=tensor_format, tensor_dtype=tensor_dtype)

OK: OpenVX using GPU device - 0: AMD Instinct MI100 [gfx908:sramecc+:xnack-] with 120 CUs on PCI bus 63:00.0

Pipeline has been created succesfully
OK: loaded 116 kernels from libvx_rpp.so


## Video Pipeline
Here we use the video reader to read the video data. Then the decoded sequences is passed to optical_flow and optical_flow_to_color.We enable the output for optical_flow_to_color using set_output.

In [5]:
with pipe:
        images = fn.readers.video(file_root=video_path, sequence_length=sequence_length,
                              random_shuffle=False, image_type=types.RGB)
        optical_image = fn.optical_flow(images)
        output_images = fn.optical_flow_to_color(optical_image)
        pipe.set_outputs(output_images)


 check in rocalOpticalFlow rocal_api_Augmentation
 check in rocalOpticalFlowToColor rocal_api_Augmentation

## Building the Pipeline
Here we are creating the pipeline. In order to use our Pipeline, we need to build it. This is achieved by calling the build function. Then iterator object is created with ROCALVideoIterator(video_pipeline)

In [6]:
# Build the pipeline
pipe.build()

In [7]:
# Dataloader
data_loader = ROCALVideoIterator(
    pipe, multiplier=pipe._multiplier, offset=pipe._offset,display=display,sequence_length=sequence_length)

____________REMAINING IMAGES____________: 330


## Visualizing outputs
We have plotted the output of the video sequence using matplotlib

In [8]:
def display_sequence(sequence):
    w = sequence.shape[1]
    h = sequence.shape[0] // sequence_length
    c = sequence.shape[2]
    columns = 3
    rows = (sequence_length + 1) // (columns)
    fig = plt.figure(figsize = (32,(16 // columns) * rows))
    gs = gridspec.GridSpec(rows, columns)
    for j in range(rows*columns):
        plt.subplot(gs[j])
        plt.axis("off")
        plt.imshow(sequence)

In [9]:
from torchvision.transforms import ToPILImage
import imageio
from PIL import Image
frames = []
for i, it in enumerate(data_loader):
    # print(it.shape)
    if i <= 6:
        for sequence in it:
            sequences = sequence.numpy()
            pil_frame = Image.fromarray(sequences.astype('uint8'))
            frames.append(pil_frame)
    # else:
output_path = 'output.gif'
frames[0].save(output_path, save_all=True, append_images=frames[1:], loop=0, duration=1)  # Adjust duration as needed
# break
data_loader.reset()

<!-- ![Image Alt Text](output.gif) -->
![SegmentLocal](output.gif "segment")
